# CS211 Final Project
## Haoyuan Pang, Ke Tian, Chongqing Gao

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import random
import math

plt.style.use('seaborn-whitegrid')

def laplace_mech(v, sensitivity, epsilon):
    return v + np.random.laplace(loc=0, scale=sensitivity / epsilon)

def laplace_mech_vec(vec, sensitivity, epsilon):
    return [v + np.random.laplace(loc=0, scale=sensitivity / epsilon) for v in vec]

def gaussian_mech(v, sensitivity, epsilon, delta):
    return v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)

def gaussian_mech_vec(vec, sensitivity, epsilon, delta):
    return [v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)
            for v in vec]

def pct_error(orig, priv):
    return np.abs(orig - priv)/orig * 100.0

df = pd.read_csv('https://raw.githubusercontent.com/HaoyuanP/CS211-Final-Project/main/cox-violent-parsed.csv')

In [2]:
# this is our dataset
df

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,...,v_type_of_assessment,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event
0,1.0,miguel hernandez,miguel,hernandez,14/08/2013,Male,18/04/1947,69,Greater than 45,Other,...,Risk of Violence,1,Low,14/08/2013,07/07/2014,14/07/2014,0,0,327,0
1,2.0,miguel hernandez,miguel,hernandez,14/08/2013,Male,18/04/1947,69,Greater than 45,Other,...,Risk of Violence,1,Low,14/08/2013,07/07/2014,14/07/2014,0,334,961,0
2,3.0,michael ryan,michael,ryan,31/12/2014,Male,06/02/1985,31,25 - 45,Caucasian,...,Risk of Violence,2,Low,31/12/2014,30/12/2014,03/01/2015,0,3,457,0
3,4.0,kevon dixon,kevon,dixon,27/01/2013,Male,22/01/1982,34,25 - 45,African-American,...,Risk of Violence,1,Low,27/01/2013,26/01/2013,05/02/2013,0,9,159,1
4,5.0,ed philo,ed,philo,14/04/2013,Male,14/05/1991,24,Less than 25,African-American,...,Risk of Violence,3,Low,14/04/2013,16/06/2013,16/06/2013,4,0,63,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18311,NaN,alexsandra beauchamps,alexsandra,beauchamps,29/12/2014,Female,21/12/1984,31,25 - 45,African-American,...,Risk of Violence,4,Low,29/12/2014,28/12/2014,07/01/2015,5,9,459,0
18312,NaN,winston gregory,winston,gregory,14/01/2014,Male,01/10/1958,57,Greater than 45,Other,...,Risk of Violence,1,Low,14/01/2014,13/01/2014,14/01/2014,0,0,808,0
18313,NaN,farrah jean,farrah,jean,09/03/2014,Female,17/11/1982,33,25 - 45,African-American,...,Risk of Violence,2,Low,09/03/2014,08/03/2014,09/03/2014,3,0,754,0
18314,NaN,florencia sanmartin,florencia,sanmartin,30/06/2014,Female,18/12/1992,23,Less than 25,Hispanic,...,Risk of Violence,4,Low,30/06/2014,15/03/2015,15/03/2015,2,0,258,0


# age + laplace

In [3]:
epsilon = 1
def age_hist():
    return  df['age'].value_counts()
age_hist() 
# age + laplace
def dp_age_hist(epsilon):
    return age_hist().apply(lambda x: laplace_mech(x,1,epsilon))  ## total privacy cost is epsilon, by parallel composition
dp_age_hist(epsilon)

22    915.506232
24    913.079642
26    911.210686
21    900.183219
25    890.604498
         ...    
83      2.626889
96      0.347730
78      1.074816
80      0.670382
79     -0.000806
Name: age, Length: 65, dtype: float64

In [4]:
# age_clipped + laplace
b=90 # age up to 90

real_sum = df['age'].sum()
def clip_age():
    clipped_age= df['age'].clip(lower=10, upper=b).sum()
    dp_clipped_age =  age_hist().apply(lambda x: laplace_mech(x,1,epsilon)) #sensitivity = 1
    return (dp_clipped_age)
clip_age()

22    913.798636
24    910.912608
26    911.679801
21    900.164449
25    890.033999
         ...    
83      0.576045
96      2.305459
78      1.461319
80      0.659178
79      1.313551
Name: age, Length: 65, dtype: float64

In [5]:
print (pct_error(age_hist(),clip_age())) 

22      0.077838
24      0.000658
26      0.104350
21      0.002365
25      0.030269
         ...    
83     48.544209
96     26.339469
78     19.503847
80    267.385862
79     78.268433
Name: age, Length: 65, dtype: float64


# Compare laplace and Gaussian mechanism in a workload in 'age' set

In [6]:
# Range Query

def range_query(df, col, a, b):
    return len(df[(df[col] >= a) & (df[col] < b)])

random_lower_bounds = [random.randint(1, 70) for _ in range(100)]
random_workload = [(lb, random.randint(lb, 100)) for lb in random_lower_bounds]
real_answers_age = [range_query(df, 'age', lb, ub) for (lb, ub) in random_workload]
real_answers_start = [range_query(df, 'start', lb, ub) for (lb, ub) in random_workload]

In [7]:
def workload_laplace_vec(workload, epsilon, col):
    ans_list = []
    L1_GS = len(workload)
    range = [range_query(df, col, work[0], work[1]) for work in workload]
    
    noise = laplace_mech_vec(range, L1_GS, epsilon)
    return noise

In [8]:
import math
def workload_gaussian_vec(workload, epsilon, delta, col):
    ans_list = []
    L2 = math.sqrt(len(workload))
    rdp = [range_query(df, col, work[0], work[1]) for work in workload]
    
    noise = gaussian_mech_vec(rdp, L2, epsilon, delta)
    return noise

In [9]:
errors_1 = [abs(r_a - l_a) for (r_a, l_a) in zip(real_answers_age, workload_laplace_vec(random_workload, 1.0, 'age'))]
print('Average absolute error for age (laplace):', np.mean(errors_1))
errors_2 = [abs(r_a - l_a) for (r_a, l_a) in zip(real_answers_age, workload_gaussian_vec(random_workload, 1.0, 1e-5, 'age'))]
print('Average absolute error for age (gaussian):', np.mean(errors_2))

Average absolute error for age (laplace): 96.0298300867582
Average absolute error for age (gaussian): 44.010144282984456


# Compare laplace and Gaussian mechanism in a workload in 'start' set


In [10]:
errors_3 = [abs(r_a - l_a) for (r_a, l_a) in zip(real_answers_start, workload_laplace_vec(random_workload, 1.0, 'start'))]
print('Average absolute error for start (laplace):', np.mean(errors_3))
errors_4 = [abs(r_a - l_a) for (r_a, l_a) in zip(real_answers_start, workload_gaussian_vec(random_workload, 1.0, 1e-5, 'start'))]
print('Average absolute error for start (gaussian):', np.mean(errors_4))

Average absolute error for start (laplace): 79.20987516013469
Average absolute error for start (gaussian): 40.490645775118686


# workload of queries on 'decile_score' columns

In [11]:
import random
def range_query(df, col, a, b):
    return len(df[(df[col] >= a) & (df[col] < b)])

random_lower_bounds = [random.randint(1, 4) for _ in range(100)]
random_workload = [(lb, random.randint(lb, 11)) for lb in random_lower_bounds]
real_answers_score = [range_query(df, 'decile_score', lb, ub) for (lb, ub) in random_workload]

In [12]:
errors_5 = [abs(r_a - l_a) for (r_a, l_a) in zip(real_answers_score, workload_laplace_vec(random_workload, 1.0, 'decile_score'))]
print('Average absolute error for decile_score (laplace):', np.mean(errors_5))
errors_6 = [abs(r_a - l_a) for (r_a, l_a) in zip(real_answers_score, workload_gaussian_vec(random_workload, 1.0, 1e-5, 'decile_score'))]
print('Average absolute error for decile_score (gaussian):', np.mean(errors_6))

Average absolute error for decile_score (laplace): 104.83235883591885
Average absolute error for decile_score (gaussian): 44.08077891153038


# Synthetic Data based on race and decile_score (connection between race and score)

For this part, I will use two different way to generate synthetic data, one is from Homework 10, the synthetic data that add two private one-way marginal, another way is using the synthetic data that use of two-way marginal. And we will store those two different synthetic data as two csv files, then use R to generate boxplot based on those two synthetic datas. 

In [13]:

score_domain = range(1,11)
race_domain = df['race'].dropna().unique()

def gen_data_one_column(col, domain, n, epsilon):
    hist = [len(df[df[col] == v]) for v in domain]
    noisy_hist = [laplace_mech(v, 1, epsilon) for v in hist]
    clipped_hist = np.clip(noisy_hist, 0, None)
    probs = clipped_hist / np.sum(clipped_hist)
    
    gen_data = [np.random.choice(domain, p=probs) for _ in range(n)]
    return gen_data


In [14]:
def gen_data_two_column(col1, domain1, col2, domain2, n, epsilon):
    data1 = gen_data_one_column(col1, domain1, n, epsilon/2)
    data2 = gen_data_one_column(col2, domain2, n, epsilon/2)
    
    return pd.DataFrame(zip(data1, data2), columns=[col1, col2])

x = gen_data_two_column('decile_score', score_domain, 'race', race_domain, 18316, 1.0)
x

,decile_score,race
0,6,African-American
1,10,African-American
2,1,Caucasian
3,2,African-American
4,6,Other
...,...,...
18311,4,Caucasian
18312,3,Other
18313,9,Other
18314,4,African-American


In [15]:
# store the data to the csv file and use R to check the relationship between decile_score and race
# this is the data without correlation
data1 = gen_data_two_column('race', race_domain, 'decile_score', score_domain, 18316, 1.0)
data1.to_csv('wrong_result.csv')


In [16]:
# two-way marginal
new_df = pd.crosstab( df['decile_score'],df['race'])
new_df

race,African-American,Asian,Caucasian,Hispanic,Native American,Other
decile_score,,,,,,
-1,12,0,10,0,0,1
1,797,27,1384,408,7,300
2,826,11,784,246,10,154
3,853,12,678,185,4,81
4,893,0,708,106,5,118
5,844,5,612,131,2,55
6,1019,5,530,101,4,54
7,1147,2,437,83,10,41
8,1138,8,385,57,2,24


In [17]:
dp_ct = new_df.applymap(lambda x: max(laplace_mech(x, 1, 1), 0))
dp_vals = dp_ct.stack().reset_index().values.tolist()
probs = [p for _,_,p in dp_vals]
vals = [(a,b) for a,b,_ in dp_vals]
probs_norm = probs / np.sum(probs)

indices = range(0, len(vals))
n = laplace_mech(len(df), 1, 1.0)
gen_indices = np.random.choice(indices, int(n), p=probs_norm)
syn_data = [vals[i] for i in gen_indices]

syn_df = pd.DataFrame(syn_data, columns=['decile_score', 'race'])
syn_df

,decile_score,race
0,1,Hispanic
1,4,Caucasian
2,7,Caucasian
3,5,African-American
4,5,African-American
...,...,...
18310,1,African-American
18311,1,African-American
18312,1,Other
18313,8,African-American


In [18]:
# store the data to the csv file and use R to check the relationship between decile_score and race
# this is the data with correlation
syn_df.to_csv('Result_corr.csv')
